# **Project Amazon Report Sales**


In [1]:
!pip install -r requirements.txt

## **Team 12**

This data set is located in https://www.kaggle.com/datasets/thedevastator/unlock-profits-with-e-commerce-sales-data?resource=download&select=Amazon+Sale+Report.csv


Write Goal here:

In [2]:
import pandas as pd
import numpy as np
import scipy as sp 
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import plotly.express as px
# suppressing warnings
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.6f}'.format

In [3]:
df = pd.read_csv('Amazon Sale Report.csv')

In [4]:
df.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.620000,MUMBAI,MAHARASHTRA,400081.000000,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.000000,BENGALURU,KARNATAKA,560085.000000,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.000000,NAVI MUMBAI,MAHARASHTRA,410210.000000,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.330000,PUDUCHERRY,PUDUCHERRY,605008.000000,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.000000,CHENNAI,TAMIL NADU,600073.000000,IN,NaN,False,NaN,NaN


## **Descriptive Analysis**

### **Number of columns and rows**

In [165]:
columns = df.columns
columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object')

In [166]:
rows = df.index
rows

RangeIndex(start=0, stop=128975, step=1)

In [167]:
df.shape

(128975, 24)

In [168]:
df.shape[0]

128975

In [169]:
df.shape[1]

24

### **Data Type**

In [170]:
df.dtypes

index                   int64
Order ID               object
Date                   object
Status                 object
Fulfilment             object
Sales Channel          object
ship-service-level     object
Style                  object
SKU                    object
Category               object
Size                   object
ASIN                   object
Courier Status         object
Qty                     int64
currency               object
Amount                float64
ship-city              object
ship-state             object
ship-postal-code      float64
ship-country           object
promotion-ids          object
B2B                      bool
fulfilled-by           object
Unnamed: 22            object
dtype: object

**Boolean Variables**

In [171]:
Boolean_Variables = list(df.select_dtypes(bool).columns)
Boolean_Variables

['B2B']

In [172]:
df['B2B'] = df['B2B'].astype(object)

**Categorical Variables**

In [173]:
df['ship-postal-code'] = df['ship-postal-code'].astype(object)
Categorical_Variables = list(df.select_dtypes(object).columns)
Categorical_Variables

['Order ID',
 'Date',
 'Status',
 'Fulfilment',
 'Sales Channel ',
 'ship-service-level',
 'Style',
 'SKU',
 'Category',
 'Size',
 'ASIN',
 'Courier Status',
 'currency',
 'ship-city',
 'ship-state',
 'ship-postal-code',
 'ship-country',
 'promotion-ids',
 'B2B',
 'fulfilled-by',
 'Unnamed: 22']

In [174]:
len(Categorical_Variables)

21

**Numerical Variables**

In [175]:
Numerical_Variables = list(set(df.columns) - set(df.select_dtypes(object).columns) - set(df.select_dtypes(bool).columns))
Numerical_Variables

['Qty', 'index', 'Amount']

### **Mean, Standard Deviation, Min and Max using Descriptive Statistics**

In [176]:
df.describe()

,index,Qty,Amount
count,128975.000000,128975.000000,121180.000000
mean,64487.000000,0.904431,648.561465
std,37232.019822,0.313354,281.211687
min,0.000000,0.000000,0.000000
25%,32243.500000,1.000000,449.000000
50%,64487.000000,1.000000,605.000000
75%,96730.500000,1.000000,788.000000
max,128974.000000,15.000000,5584.000000


### **Median**

In [177]:
pd.DataFrame(df[Numerical_Variables].median()).rename(columns={0:'Median'})

,Median
Qty,1.000000
index,64487.000000
Amount,605.000000


### **Mode**

In [178]:
df.mode()[:1].T.rename(columns={0:'Mode'})

,Mode
index,0
Order ID,171-5057375-2831560
Date,05-03-22
Status,Shipped
Fulfilment,Amazon
Sales Channel,Amazon.in
ship-service-level,Expedited
Style,JNE3797
SKU,JNE3797-KR-L
Category,Set


### **Variance**

In [179]:
pd.DataFrame(df[Numerical_Variables].var()).rename(columns={0:'Variance'})

,Variance
Qty,0.098190
index,1386223300.000000
Amount,79080.013034


### **Min**

In [180]:
pd.DataFrame(df.min()).rename(columns={0:'Min'})

,Min
index,0
Order ID,171-0000547-8192359
Date,03-31-22
Status,Cancelled
Fulfilment,Amazon
Sales Channel,Amazon.in
ship-service-level,Expedited
Style,AN201
SKU,AN201-RED-M
Category,Blouse


### **Null Values**

In [181]:
pd.DataFrame(df.isnull().sum()).rename(columns={0:'Null Values'})

,Null Values
index,0
Order ID,0
Date,0
Status,0
Fulfilment,0
Sales Channel,0
ship-service-level,0
Style,0
SKU,0
Category,0


### **Percentage of Null Values**

In [182]:
pd.DataFrame(df.isnull().sum() * 100 / len(df)).rename(columns={0:'% Null Values'})

,% Null Values
index,0.000000
Order ID,0.000000
Date,0.000000
Status,0.000000
Fulfilment,0.000000
Sales Channel,0.000000
ship-service-level,0.000000
Style,0.000000
SKU,0.000000
Category,0.000000


### **Correlation Matrix**

In [183]:
df.corr()

,index,Qty,Amount
index,1.000000,0.010621,0.047571
Qty,0.010621,1.000000,0.066900
Amount,0.047571,0.066900,1.000000


## **Data Cleaning and Precessinng**

### **Table of df type, null values and unique values for better visualization**

In [240]:
def printinfo():
    temp = pd.DataFrame(index= df.columns)
    temp['data_type'] = df.dtypes
    temp['null_count'] =df.isnull().sum()
    temp['unique_count'] = df.nunique()
    return temp
printinfo()

,data_type,null_count,unique_count
Status,int64,0,2
Fulfilment,object,0,2
Sales Channel,object,0,2
ship-service-level,object,0,2
Category,object,0,9
Size,object,0,11
Courier Status,object,0,3
ship-state,object,0,69
B2B,object,0,2
Qty_standardized,float64,0,10


## **Dropping Variables**

### Dropping Numerical Variables Zero variance

In [185]:
df[Numerical_Variables].std()

Qty          0.313354
index    37232.019822
Amount     281.211687
dtype: float64

In [186]:
df = df.drop(df[Numerical_Variables].std()[df[Numerical_Variables].std()== 0].index, axis = 1) 

In [187]:
df[Numerical_Variables].std()

Qty          0.313354
index    37232.019822
Amount     281.211687
dtype: float64

In [188]:
df = df.drop('index', axis = 1)

In [189]:
df.head()

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,...,INR,647.620000,MUMBAI,MAHARASHTRA,400081.000000,IN,NaN,False,Easy Ship,NaN
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,...,INR,406.000000,BENGALURU,KARNATAKA,560085.000000,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,...,INR,329.000000,NAVI MUMBAI,MAHARASHTRA,410210.000000,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,...,INR,753.330000,PUDUCHERRY,PUDUCHERRY,605008.000000,IN,NaN,False,Easy Ship,NaN
4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,...,INR,574.000000,CHENNAI,TAMIL NADU,600073.000000,IN,NaN,False,NaN,NaN


### Dropping Categorical Variables with Zero variance

In [190]:
Categorical_Variables = list(df.select_dtypes(object).columns)
Categorical_Variables

['Order ID',
 'Date',
 'Status',
 'Fulfilment',
 'Sales Channel ',
 'ship-service-level',
 'Style',
 'SKU',
 'Category',
 'Size',
 'ASIN',
 'Courier Status',
 'currency',
 'ship-city',
 'ship-state',
 'ship-postal-code',
 'ship-country',
 'promotion-ids',
 'B2B',
 'fulfilled-by',
 'Unnamed: 22']

In [191]:
len(Categorical_Variables)

21

In [192]:
zero_cardinality = [] 

for i in Categorical_Variables: 
    if len(df[i].value_counts().index) == 1:
        zero_cardinality.append(i)
zero_cardinality

['currency', 'ship-country', 'fulfilled-by', 'Unnamed: 22']

In [193]:
df = df.drop(zero_cardinality, axis = 1)

In [194]:
df.head()

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,promotion-ids,B2B
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,647.620000,MUMBAI,MAHARASHTRA,400081.000000,NaN,False
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,406.000000,BENGALURU,KARNATAKA,560085.000000,Amazon PLCC Free-Financing Universal Merchant ...,False
2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,329.000000,NAVI MUMBAI,MAHARASHTRA,410210.000000,IN Core Free Shipping 2015/04/08 23-48-5-108,True
3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,753.330000,PUDUCHERRY,PUDUCHERRY,605008.000000,NaN,False
4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,574.000000,CHENNAI,TAMIL NADU,600073.000000,NaN,False


### Dropping Categorical Variables with Many Levels

In [195]:
Categorical_Variables = list(df.select_dtypes(object).columns)
Categorical_Variables

['Order ID',
 'Date',
 'Status',
 'Fulfilment',
 'Sales Channel ',
 'ship-service-level',
 'Style',
 'SKU',
 'Category',
 'Size',
 'ASIN',
 'Courier Status',
 'ship-city',
 'ship-state',
 'ship-postal-code',
 'promotion-ids',
 'B2B']

In [196]:
len(Categorical_Variables)

17

In [197]:
high_cardinality = [] 

for i in Categorical_Variables: # for each categorical variables
    if len(df[i].value_counts().index) > 200: # check how many levels it has and if it is more
        high_cardinality.append(i) # than 200, variable has many levels
        # so append it to the list of categorical variables with high cardinality
        
print(high_cardinality) # Prints the list of vari

['Order ID', 'Style', 'SKU', 'ASIN', 'ship-city', 'ship-postal-code', 'promotion-ids']


In [198]:
df = df.drop(high_cardinality, axis = 1)

In [199]:
df.head()

,Date,Status,Fulfilment,Sales Channel,ship-service-level,Category,Size,Courier Status,Qty,Amount,ship-state,B2B
0,04-30-22,Cancelled,Merchant,Amazon.in,Standard,Set,S,NaN,0,647.620000,MAHARASHTRA,False
1,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,kurta,3XL,Shipped,1,406.000000,KARNATAKA,False
2,04-30-22,Shipped,Amazon,Amazon.in,Expedited,kurta,XL,Shipped,1,329.000000,MAHARASHTRA,True
3,04-30-22,Cancelled,Merchant,Amazon.in,Standard,Western Dress,L,NaN,0,753.330000,PUDUCHERRY,False
4,04-30-22,Shipped,Amazon,Amazon.in,Expedited,Top,3XL,Shipped,1,574.000000,TAMIL NADU,False


## Data Imputation

**Numerical Variables**

In [200]:
df.isnull().sum()

Date                     0
Status                   0
Fulfilment               0
Sales Channel            0
ship-service-level       0
Category                 0
Size                     0
Courier Status        6872
Qty                      0
Amount                7795
ship-state              33
B2B                      0
dtype: int64

In [201]:
Numerical_Variables = list(df.select_dtypes(exclude = object).columns)
Numerical_Variables

['Qty', 'Amount']

In [202]:
df['Amount'].median()

605.0

In [203]:
df['Amount'] = df['Amount'].fillna(df['Amount'].median(), inplace = False)

In [204]:
df.isnull().sum()

Date                     0
Status                   0
Fulfilment               0
Sales Channel            0
ship-service-level       0
Category                 0
Size                     0
Courier Status        6872
Qty                      0
Amount                   0
ship-state              33
B2B                      0
dtype: int64

**Categorial Variable**

In [205]:
Categorical_Variables = list(df.select_dtypes(object).columns)
Categorical_Variables

['Date',
 'Status',
 'Fulfilment',
 'Sales Channel ',
 'ship-service-level',
 'Category',
 'Size',
 'Courier Status',
 'ship-state',
 'B2B']

In [206]:
df[Categorical_Variables].mode()

,Date,Status,Fulfilment,Sales Channel,ship-service-level,Category,Size,Courier Status,ship-state,B2B
0,05-03-22,Shipped,Amazon,Amazon.in,Expedited,Set,M,Shipped,MAHARASHTRA,False


In [207]:
df = df.fillna({'Courier Status' : 'Shipped', 'ship-state' : 'MAHARASHTRA'})

In [208]:
df.isnull().sum()

Date                  0
Status                0
Fulfilment            0
Sales Channel         0
ship-service-level    0
Category              0
Size                  0
Courier Status        0
Qty                   0
Amount                0
ship-state            0
B2B                   0
dtype: int64

##  Standardization

**Numerical Variables**

In [209]:
Numerical_Variables = list(df.select_dtypes(exclude = object).columns)
Numerical_Variables

['Qty', 'Amount']

In [210]:
new_col = [i+'_standardized' for i in Numerical_Variables] 
new_col

['Qty_standardized', 'Amount_standardized']

In [211]:
array = df[Numerical_Variables].values 
array

array([[0.0000e+00, 6.4762e+02],
       [1.0000e+00, 4.0600e+02],
       [1.0000e+00, 3.2900e+02],
       ...,
       [1.0000e+00, 6.9000e+02],
       [1.0000e+00, 1.1990e+03],
       [1.0000e+00, 6.9600e+02]])

In [212]:
data_scaler = StandardScaler().fit(array) 
df[new_col] = pd.DataFrame(data_scaler.transform(array))
df = df.drop(Numerical_Variables, axis = 1)
df.head()

,Date,Status,Fulfilment,Sales Channel,ship-service-level,Category,Size,Courier Status,ship-state,B2B,Qty_standardized,Amount_standardized
0,04-30-22,Cancelled,Merchant,Amazon.in,Standard,Set,S,Shipped,MAHARASHTRA,False,-2.886307,0.006200
1,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,kurta,3XL,Shipped,KARNATAKA,False,0.304989,-0.879576
2,04-30-22,Shipped,Amazon,Amazon.in,Expedited,kurta,XL,Shipped,MAHARASHTRA,True,0.304989,-1.161857
3,04-30-22,Cancelled,Merchant,Amazon.in,Standard,Western Dress,L,Shipped,PUDUCHERRY,False,-2.886307,0.393732
4,04-30-22,Shipped,Amazon,Amazon.in,Expedited,Top,3XL,Shipped,TAMIL NADU,False,0.304989,-0.263690


## Extracting year, month and day from pickup_datetime and dropoff_datetime

In [213]:
import datetime
df['Date'] = pd.to_datetime(df['Date'], format= '%m-%d-%y')

In [214]:
df['Purcahase_Year'] = df['Date'].dt.year
df['Purcahase_Month'] = df['Date'].dt.month
df['Purcahase_Day']= df['Date'].dt.day

In [215]:
df.head()

,Date,Status,Fulfilment,Sales Channel,ship-service-level,Category,Size,Courier Status,ship-state,B2B,Qty_standardized,Amount_standardized,Purcahase_Year,Purcahase_Month,Purcahase_Day
0,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,Set,S,Shipped,MAHARASHTRA,False,-2.886307,0.006200,2022,4,30
1,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,kurta,3XL,Shipped,KARNATAKA,False,0.304989,-0.879576,2022,4,30
2,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,kurta,XL,Shipped,MAHARASHTRA,True,0.304989,-1.161857,2022,4,30
3,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,Western Dress,L,Shipped,PUDUCHERRY,False,-2.886307,0.393732,2022,4,30
4,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,Top,3XL,Shipped,TAMIL NADU,False,0.304989,-0.263690,2022,4,30


In [216]:
df = df.drop(['Date' ], axis =1 )

In [217]:
df.head()

,Status,Fulfilment,Sales Channel,ship-service-level,Category,Size,Courier Status,ship-state,B2B,Qty_standardized,Amount_standardized,Purcahase_Year,Purcahase_Month,Purcahase_Day
0,Cancelled,Merchant,Amazon.in,Standard,Set,S,Shipped,MAHARASHTRA,False,-2.886307,0.006200,2022,4,30
1,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,kurta,3XL,Shipped,KARNATAKA,False,0.304989,-0.879576,2022,4,30
2,Shipped,Amazon,Amazon.in,Expedited,kurta,XL,Shipped,MAHARASHTRA,True,0.304989,-1.161857,2022,4,30
3,Cancelled,Merchant,Amazon.in,Standard,Western Dress,L,Shipped,PUDUCHERRY,False,-2.886307,0.393732,2022,4,30
4,Shipped,Amazon,Amazon.in,Expedited,Top,3XL,Shipped,TAMIL NADU,False,0.304989,-0.263690,2022,4,30


## Encoding Categorical Variables

In [218]:
Categorical_Variables = list(df.dtypes[df.dtypes == object].index)

In [219]:
df['Status'].value_counts() #Variable to study to determinate if a product will be shipped to the buyer or it will return
#for the model we will need just 2 outcome thats why we are aggrouping them

Shipped                          77804
Shipped - Delivered to Buyer     28769
Cancelled                        18332
Shipped - Returned to Seller      1953
Shipped - Picked Up                973
Pending                            658
Pending - Waiting for Pick Up      281
Shipped - Returning to Seller      145
Shipped - Out for Delivery          35
Shipped - Rejected by Buyer         11
Shipping                             8
Shipped - Lost in Transit            5
Shipped - Damaged                    1
Name: Status, dtype: int64

In [220]:
df['Status'] = df['Status'].replace('Shipped - Delivered to Buyer', 'Shipped')
df['Status'] = df['Status'].replace('Shipped - Returned to Seller', 'Cancelled')
df['Status'] = df['Status'].replace('Shipped - Picked Up', 'Shipped')
df['Status'] = df['Status'].replace('Pending', 'Cancelled')
df['Status'] = df['Status'].replace('Pending - Waiting for Pick Up', 'Cancelled')
df['Status'] = df['Status'].replace('Shipped - Returning to Seller', 'Cancelled')
df['Status'] = df['Status'].replace('Shipped - Out for Delivery', 'Cancelled')
df['Status'] = df['Status'].replace('Shipped - Rejected by Buyer', 'Cancelled')
df['Status'] = df['Status'].replace('Shipping', 'Cancelled')
df['Status'] = df['Status'].replace('Shipped - Lost in Transit', 'Cancelled')
df['Status'] = df['Status'].replace('Shipped - Damaged', 'Cancelled')

In [221]:
df['Status'].value_counts()

Shipped      107546
Cancelled     21429
Name: Status, dtype: int64

In [222]:
df['Status'] = df['Status'].replace('Shipped', 1)
df['Status'] = df['Status'].replace('Cancelled', 0)

In [223]:
df.head()

,Status,Fulfilment,Sales Channel,ship-service-level,Category,Size,Courier Status,ship-state,B2B,Qty_standardized,Amount_standardized,Purcahase_Year,Purcahase_Month,Purcahase_Day
0,0,Merchant,Amazon.in,Standard,Set,S,Shipped,MAHARASHTRA,False,-2.886307,0.006200,2022,4,30
1,1,Merchant,Amazon.in,Standard,kurta,3XL,Shipped,KARNATAKA,False,0.304989,-0.879576,2022,4,30
2,1,Amazon,Amazon.in,Expedited,kurta,XL,Shipped,MAHARASHTRA,True,0.304989,-1.161857,2022,4,30
3,0,Merchant,Amazon.in,Standard,Western Dress,L,Shipped,PUDUCHERRY,False,-2.886307,0.393732,2022,4,30
4,1,Amazon,Amazon.in,Expedited,Top,3XL,Shipped,TAMIL NADU,False,0.304989,-0.263690,2022,4,30


In [224]:
X = df.drop(['Status'], axis=1)
y = df['Status']

In [225]:
#Normalization

In [226]:
Categorical_Variables.remove('Status')

In [227]:
# Create dummy variables using onehot encoding
dummy_cat_df = pd.get_dummies(X[Categorical_Variables], drop_first=True) 

In [228]:
# Drops categorical variables from the df
X = X.drop(Categorical_Variables, axis = 1) 

# Adds the newly created dummy variables instead
X = pd.concat([X, dummy_cat_df], axis = 1) 
X

,Qty_standardized,Amount_standardized,Purcahase_Year,Purcahase_Month,Purcahase_Day,Fulfilment_Merchant,Sales Channel _Non-Amazon,ship-service-level_Standard,Category_Bottom,Category_Dupatta,...,ship-state_UTTARAKHAND,ship-state_WEST BENGAL,ship-state_bihar,ship-state_delhi,ship-state_goa,ship-state_orissa,ship-state_punjab,ship-state_rajasthan,ship-state_rajsthan,B2B_True
0,-2.886307,0.006200,2022,4,30,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.304989,-0.879576,2022,4,30,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.304989,-1.161857,2022,4,30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,-2.886307,0.393732,2022,4,30,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.304989,-0.263690,2022,4,30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128970,0.304989,-0.472651,2022,5,31,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128971,0.304989,1.294355,2022,5,31,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128972,0.304989,0.161565,2022,5,31,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128973,0.304989,2.027553,2022,5,31,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [229]:
#PCA

In [230]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
pca.fit(X)
x_pca = pca.transform(X)

In [231]:
#Splitting data int training and testing

In [232]:
from sklearn.model_selection import train_test_split

In [233]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [234]:
#Building LogisticRegression

In [235]:
from sklearn.linear_model import LogisticRegression

# Create a logistic regression object
logreg = LogisticRegression()

# Fit the logistic regression model to the training data
logreg.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = logreg.predict(X_test)


In [236]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate the precision
precision = precision_score(y_test, y_pred)

# Calculate the recall
recall = recall_score(y_test, y_pred)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))
print("Precision: {:.2f}%".format(precision*100))
print("Recall: {:.2f}%".format(recall*100))
print("F1 score: {:.2f}%".format(f1*100))


Accuracy: 98.12%
Precision: 97.97%
Recall: 99.81%
F1 score: 98.88%


In [237]:
#Building RandomForest

In [238]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier object
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the classifier to the training data
rf.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = rf.predict(X_test)

In [239]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate the precision
precision = precision_score(y_test, y_pred)

# Calculate the recall
recall = recall_score(y_test, y_pred)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))
print("Precision: {:.2f}%".format(precision*100))
print("Recall: {:.2f}%".format(recall*100))
print("F1 score: {:.2f}%".format(f1*100))


Accuracy: 98.01%
Precision: 98.04%
Recall: 99.61%
F1 score: 98.82%
